# 运动边界非定常流场预测

## 环境安装

本案例要求 **MindSpore >= 2.0.0** 版本以调用如下接口: *mindspore.jit, mindspore.jit_class, mindspore.data_sink*。具体请查看[MindSpore安装](https://www.mindspore.cn/install)。

此外，你需要安装 **MindFlow >=0.1.0** 版本。如果当前环境还没有安装，请按照下列方式选择后端和版本进行安装。

In [ ]:
mindflow_version = "0.1.0"  # update if needed
# GPU Comment out the following code if you are using NPU.
!pip uninstall -y mindflow-gpu
!pip install mindflow-gpu==$mindflow_version

# NPU Uncomment if needed.
# !pip uninstall -y mindflow-ascend
# !pip install mindflow-ascend==$mindflow_version

## 概述

CFD作为一种通过数值方法来模拟和解析流体运动的重要工具，极大便利了流体力学相关问题的科学研究，在设计、优化和研究领域提供准确的数据和见解并发挥着重要作用。流体力学中具有代表性和研究价值的一类问题是：对具有移动边界的非定常流场系统进行模拟，以分析运动结构在流场中的受力情况，可在工程上优化设计运动结构，为航空航天飞行器以及航海器等外形优化提供方案策略。高精确度计算流体力学技术（CFD）能够准确模拟流场演化和结构受力情况，但是高精度动边界问题需要大量网格，导致硬件消耗和计算时间成本巨大，另外对动态网格的构造也格外耗时。

面对CFD在应用于复杂问题时计算量巨大并且计算精度有待提高等问题，智能流体力学领域给出了行之有效的解决方案，深度学习可以通过深度神经网络可学习流动工况与流场之间的演化关系，快速实现流场高精度预测与重构。为了高效解决动边界流场重构问题，提出了一种混合深度神经网络(HDNN)，以实现非定常动边界流场重构，并基于此实现流场快速预测。

## 问题描述

流场相关尺寸如图所示，其中 $Y = Asin(2πft)$ 代表圆柱体在竖直方向做简谐运动的运动表达式，A为振幅，f为频率；D代表圆柱体直径；矩形边界代表计算域。均匀来流流过运动圆柱体时，在流体与固体相互作用的影响下，会在圆柱体后方形成一系列复杂的流动现象，如边界层分离、交替出现的卡门涡街等，并演化为物理量随时间周期性变化的非均匀流场。

## 技术路径

MindFlow求解该问题的具体流程如下：

1.根据CFD数值模拟结果创建数据集。

2.使用MindSpore深度学习框架构建模型。

3.定义优化器与损失函数。

4.使用MindSpore的即时编译等加速模型训练。

5.利用训练好的模型进行推理和可视化。

![p1.png](./images/p1.png)

## 模型架构

HDNN的基本框架由卷积神经网络（CNN）、卷积长短期记忆网络（ConvLSTM）和反卷积神经网络（DeCNN）组成。CNN降低了时间序列流场的维数，实现特征提取；ConvLSTM学习低维时空特征并进行预测；最后，DeCNN实现预测流场的重建

+ 输入层：输入历史流场
+ 卷积层：通过多层CNN对输入流场进行降维，提取高维时空流动特征
+ 记忆层：通过ConvLSTM学习低维空间流场时空特征的演变，预测下一时刻
+ 反卷积输出层：将预测流场的低维特征恢复到高维空间，通过多层DeCNN重构下一时刻的瞬态流场，并输出可视化预测结果

![HDNN.jpg](./images/HDNN.jpg)

## 训练数据集

数据集由非定常二维圆柱绕流的数值仿真流场数据构建的多维矩阵流场快照矩阵构建而成

+ 二维圆柱在均匀来流流场中做一维简谐振动，振动频率f（Hz）分别为1.25、1.43、1.67、2.00，振幅比A/D分别为0.5、0.6、0.7、0.8。两两组合总共16组运动状态
+ 数据集为某一状态（f,A/D）下的非定常流场序列数据
+ 每张流场快照包含3个通道，代表流场的压强分布信息、水平速度信息、竖直速度信息，多维矩阵流场快照矩阵尺寸为：T×C×H×W(C为通道数，H，W分别为快照的高和宽）
+ 数据集：[下载位置](https://download.mindspore.cn/mindscience/mindflow/dataset/applications/data_driven/move_boundary_hdnn)

In [1]:
import os
import time
import argparse
import numpy as np

from mindspore import nn, ops, context, save_checkpoint, set_seed, data_sink, jit
from mindflow.utils import load_yaml_config

from src import my_train_dataset, AEnet, save_loss_curve

## 训练环境

+ 训练采用Mindspore框架的静态图模式（GRAPH）
+ 在CPU、GPU或Ascend进行训练（单卡）
+ 训练数据集中的圆柱振动频率f（Hz）分别为1.25、1.43、1.67，振幅比A/D分别为0.5、0.6、0.7。两两组合总共9组运动状态

In [2]:
set_seed(0)
np.random.seed(0)

## 训练超参数

从config中获得模型、数据、优化器的超参

In [3]:
parser = argparse.ArgumentParser(description="cylinder around flow ROM")

parser.add_argument("--mode", type=str, default="GRAPH", choices=["GRAPH", "PYNATIVE"],
                    help="Context mode, support 'GRAPH', 'PYNATIVE'")
parser.add_argument("--save_graphs", type=bool, default=False, choices=[True, False],
                    help="Whether to save intermediate compilation graphs")
parser.add_argument("--save_graphs_path", type=str, default="./summary")
parser.add_argument("--device_target", type=str, default="GPU", choices=["GPU", "Ascend"],
                    help="The target device to run, support 'GPU','Ascend'")
parser.add_argument("--device_id", type=int, default=0, help="ID of the target device")
parser.add_argument("--data_list", type=list, default=['0.00', '0.25', '0.35', '0.45'],
                    help="The type for training, [0.00, 0.25, 0.35, 0.45] for multi_state training /n"
                         "[0.25],....,[0.45] for single_state training")
parser.add_argument('--batch_size', type=int, default=16, help="mini batch_size")
parser.add_argument("--config_file_path", type=str, default="./config.yaml")

args = parser.parse_args()

context.set_context(mode=context.GRAPH_MODE if args.mode.upper().startswith("GRAPH") else context.PYNATIVE_MODE,
                    save_graphs=args.save_graphs, save_graphs_path=args.save_graphs_path,
                    device_target=args.device_target, device_id=args.device_id)
use_ascend = context.get_context(attr_key='device_target') == "Ascend"

config = load_yaml_config(args.config_file_path)
data_params = config["data"]
model_params = config["model"]
optimizer_params = config["optimizer"]

## 训练过程文件保存路径

将训练好的模型文件每隔一定训练次数保存在文件夹下

In [4]:
ckpt_dir = optimizer_params["ckpt_dir"]
if not os.path.exists(ckpt_dir):
    os.mkdir(ckpt_dir)

## 构建神经网络及优化器

神经网络的卷积层共有12层，ConvLSTM有1层，反卷积共有12层

损失函数使用均方误差（Mean Squared Error）损失函数，优化器使用Adam（Adaptive Moment Estimation）优化算法

In [5]:
model = AEnet(in_channels=model_params["in_channels"],
              num_layers=model_params["num_layers"],
              kernel_size=model_params["kernel_size"],
              num_convlstm_layers=model_params["num_convlstm_layers"])

loss_func = nn.MSELoss()
optimizer = nn.Adam(params=model.trainable_params(), learning_rate=optimizer_params["lr"])
if use_ascend:
    from mindspore.amp import DynamicLossScaler, auto_mixed_precision, all_finite
    loss_scaler = DynamicLossScaler(1024, 2, 100)
    auto_mixed_precision(model, 'O1')
else:
    loss_scaler = None

## 训练框架

定义前向传播函数forward_fn，将预测值和真值比较得到损失值loss并返回

In [6]:
def forward_fn(inputs, velocity, label):
    pred = model(inputs, velocity)
    loss = loss_func(pred, label)

    if use_ascend:
        loss = loss_scaler.scale(loss)
    return loss

grad_fn = ops.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=False)

## 数据集加载

给my_train_dataset传参，得到训练数据集和验证数据集

In [7]:
print(f"==================Load data sample ===================")
dataset_train, dataset_eval = my_train_dataset(data_params["data_dir"],
                                               data_params["time_steps"],
                                               args.data_list)
print(f"======================End Load========================\n")

## 数据下沉及模型训练

定义train_step和eval_step并使用data_sink加速训练，输出训练过程的损失值和使用时间，并每隔一定训练轮次保存模型文件

In [8]:
print(f"====================Start train=======================")
@jit
def train_step(inputs, velocity, label):
    loss, grads = grad_fn(inputs, velocity, label)
    if use_ascend:
        loss = loss_scaler.unscale(loss)
        if all_finite(grads):
            grads = loss_scaler.unscale(grads)
    loss = ops.depend(loss, optimizer(grads))
    return loss

@jit
def eval_step(inputs, velocity, label):
    loss = forward_fn(inputs, velocity, label)
    loss = ops.sqrt(loss)
    return loss

train_sink_process = data_sink(train_step, dataset_train, sink_size=1)
eval_sink_process = data_sink(eval_step, dataset_eval, sink_size=1)
train_data_size, eval_data_size = dataset_train.get_dataset_size(), dataset_eval.get_dataset_size()

avg_train_losses = []
avg_valid_losses = []

for epoch in range(1, optimizer_params["epochs"] + 1):
    train_losses = 0
    valid_losses = 0

    local_time_beg = time.time()
    model.set_train(True)

    for _ in range(train_data_size):
        step_train_loss = ops.squeeze(train_sink_process(), axis=())
        step_train_loss = step_train_loss.asnumpy().item()
        train_losses += step_train_loss

    train_loss = train_losses / train_data_size
    avg_train_losses.append(train_loss)

    print(f"epoch: {epoch}, epoch average train loss: {train_loss :.6f}, "
          f"epoch time: {(time.time() - local_time_beg):.2f}s")

    if epoch % optimizer_params["eval_interval"] == 0:
        print(f"=================Start Evaluation=====================")

        eval_time_beg = time.time()
        model.set_train(False)
        for _ in range(eval_data_size):
            step_eval_loss = ops.squeeze(eval_sink_process(), axis=())
            step_eval_loss = step_eval_loss.asnumpy().item()
            valid_losses += step_eval_loss

        valid_loss = valid_losses / eval_data_size
        avg_valid_losses.append(valid_loss)

        print(f"epoch: {epoch}, epoch average valid loss: {valid_loss :.6f}, "
              f"epoch time: {(time.time() - eval_time_beg):.2f}s")
        print(f"==================End Evaluation======================")

    if epoch % optimizer_params["save_ckpt_interval"] == 0:
        save_checkpoint(model, f"{ckpt_dir}/net_{epoch}.ckpt")

save_loss_curve(avg_train_losses, 'Epoch', 'avg_train_losses', 'Avg_train_losses Curve', 'Avg_train_losses.png')
save_loss_curve(avg_valid_losses, 'Epoch', 'avg_valid_losses', 'Avg_valid_losses Curve', 'Avg_valid_losses.png')

print(f"=====================End train========================")

## 设置训练条件 传参

当运行该文件时，通过参数解析器传入必要参数，开始训练，并打印进程和设备id，以及训练总时间

In [9]:
if __name__ == "__main__":
    print("Process ID:", os.getpid())
    print(f"device id: {args.device_id}")
    start_time = time.time()
    train()
    print(f"End-to-End total time: {(time.time() - start_time):.2f}s")

## 预测流场结果可视化

+ 动边界流场预测通过执行eval.py开始预测，分为两种预测方式：单步流场预测（infer_mode为"one"）和一个振动周期内连续流场预测（infer_mode为"cycle"）；单步流场预测仅预测下一时刻一个时间步长的流场，连续流场预测则持续预测一个完整周期的流场
+ 下图为训练完备的HDNN模型实现对振动频率为1.43Hz，振幅为0.8（振幅比泛化状态）下非定常动边界单步预测和一完整周期预测的结果（展示压强场、水平速度场和竖直速度场）

![pred_single_step_puv.jpg](./images/pred_single_step_puv.jpg)

![pred_cycle_puv.jpg](./images/pred_cycle_puv.jpg)